In [1]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account a

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "jkkn/finetuned_llama3_text_to_cypher", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

## Model Evaluation on a Dataset

In [5]:
from datasets import load_dataset

## Format the dataset
def formatting_prompts_func(examples):
    texts = []
    for question, cypher, schema in zip(examples['question'], examples['cypher'], examples['schema']):
        convo = [
            {"role": "system", "content": "Given an input question, convert it to a Cypher query. No pre-amble."},
            {"role": "user", "content": f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
{schema}

Question: {question}
Cypher query:"""},
            {"role": "assistant", "content": f"{cypher}"}
        ]
        text = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False)
        texts.append(text)
    return { "text" : texts }


dataset = load_dataset("tomasonjo/text2cypher-gpt4o-clean", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/7735 [00:00<?, ? examples/s]

Map:   0%|          | 0/7735 [00:00<?, ? examples/s]

In [6]:
def generate_cypher_query_on_test_dataset(df):
    cypher_queries = []
    i=0
    for idx, record in df.iterrows():
        if i%10 == 0:
            print(f"Processed {i} records")

        i+=1
        messages = [
            {"role": "system", "content": "Given an input question, convert it to a Cypher query. No pre-amble."},
            {"role": "user", "content": f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question

            Schema: {record['schema']}

            Question: {record['question']}
            Cypher query:"""}
        ]

        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True, # Must add for generation
            return_tensors="pt",
        ).to("cuda")

        outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
        result = tokenizer.batch_decode(outputs)
        cypher_query = result[0].split("Cypher query:")[-1].strip().split("\n\n")[-1].split("<|eot_id|>")[0]
        cypher_queries.append(cypher_query)

    return cypher_queries

* Select a Sample of Records

In [7]:
n_records = 30  # Adjust the number of records as needed

df = dataset.to_pandas()

# Select a sample of records
sample_test_df = df.sample(n=n_records, random_state=42)  # Adjust n as needed

sample_test_df.head()

,question,cypher,database,schema,text
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...
3914,Show the first 5 movies that were originally t...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...
3656,Name the top 5 movies with the longest runtime.,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...
6124,What are the top 5 movies with the most distin...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,<|begin_of_text|><|start_header_id|>system<|en...


### Generate the Cypher Queries on USER Question

In [8]:
# Generate Cypher queries for the sample records
test_data_generated_queries = generate_cypher_query_on_test_dataset(sample_test_df)

len(test_data_generated_queries)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processed 0 records
Processed 10 records
Processed 20 records


30

In [9]:
test_data_generated_queries[0]

'MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE f.amount > 10000000000\nRETURN c.name AS country, COUNT(f) AS filingCount\nORDER BY filingCount DESC\nLIMIT 5'

In [10]:
sample_test_df['model_generated_cypher_queries'] = test_data_generated_queries
sample_test_df.head()

,question,cypher,database,schema,text,model_generated_cypher_queries
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE ...
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...
3914,Show the first 5 movies that were originally t...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...
3656,Name the top 5 movies with the longest runtime.,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR..."
6124,What are the top 5 movies with the most distin...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...


In [11]:
sample_test_df['generated_queries_match'] = sample_test_df['cypher'] == sample_test_df['model_generated_cypher_queries']
sample_test_df.head()

,question,cypher,database,schema,text,model_generated_cypher_queries,generated_queries_match
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE ...,False
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,False
3914,Show the first 5 movies that were originally t...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,False
3656,Name the top 5 movies with the longest runtime.,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",True
6124,What are the top 5 movies with the most distin...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,False


## Observations:

```
Question: Name the first 3 movies with a released year divisible by 10.

### Acutal Cypher Query:

    MATCH (m:Movie)
    WHERE m.released % 10 = 0
    RETURN m.title
    ORDER BY m.released
    LIMIT 3

### Model Generated Cypher:

    MATCH (m:Movie)
    WHERE m.released % 10 = 0
    RETURN m.title
    LIMIT 3
```


```
Q: Which three users have the highest average rating given to movies?

### Acutal Cypher Query:

    MATCH (u:User)-[r:RATED]->(m:Movie)
    WITH u, AVG(r.rating) AS avgRating
    ORDER BY avgRating DESC
    LIMIT 3
    RETURN u.name, avgRating


### Model Generated Cypher:

    MATCH (u:User)-[r:RATED]->(m:Movie)
    WITH u, avg(r.rating) AS avg_rating
    ORDER BY avg_rating DESC
    LIMIT 3
    RETURN u.name AS user_name, avg_rating
```

* The Model generated response are not bad, only difference is order based or returing few other properties. so, comparing string-to-string is not an ideal approach, lets use similarity based approach to compare the actual and model generated queries.

In [13]:
## Lets verify few records
for idx, row in sample_test_df.sample(5).iterrows():
    print(f"Q: {row['question']}\n\nCypher Query:\n{row['cypher']}\n\nModel Generated Cypher:\n{row['model_generated_cypher_queries']}\n")


Q: Which 3 filings have the most significant difference in latitude between originator and beneficiary banks?

Cypher Query:
MATCH (f:Filing)
WITH f, 
     toFloat(f.origin_lat) AS origin_lat, 
     toFloat(f.beneficiary_lat) AS beneficiary_lat, 
     abs(toFloat(f.origin_lat) - toFloat(f.beneficiary_lat)) AS lat_diff
ORDER BY lat_diff DESC
LIMIT 3
RETURN f.sar_id AS filing_id, f.originator_bank AS originator_bank, f.beneficiary_bank AS beneficiary_bank, lat_diff

Model Generated Cypher:
MATCH (f:Filing)-[:ORIGINATOR]->(o:Entity)-[:COUNTRY]->(oc:Country {location: point({latitude: o.origin_lat, longitude: o.origin_lng})})-[:FILED]->(f1)
MATCH (f:Filing)-[:BENEFITS]->(

Q: Name the first 3 movies with a released year divisible by 10.

Cypher Query:
MATCH (m:Movie)
WHERE m.released % 10 = 0
RETURN m.title
ORDER BY m.released
LIMIT 3

Model Generated Cypher:
MATCH (m:Movie)
WHERE m.released % 10 = 0
RETURN m.title
LIMIT 3

Q: Which three users have the highest average rating given to movi

In [61]:
# !pip install sentence_transformers

In [16]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
similarity_comparison_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
sample_test_df.head()

,question,cypher,database,schema,text,model_generated_cypher_queries,generated_queries_match
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE ...,False
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,False
3914,Show the first 5 movies that were originally t...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,False
3656,Name the top 5 movies with the longest runtime.,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",True
6124,What are the top 5 movies with the most distin...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,False


In [51]:
def generate_similarity_scores(df, model):
  scores = []
  for idx, row in df.iterrows():
    scores.append(model.similarity(model.encode(row['cypher']), model.encode(row['model_generated_cypher_queries']))[0][0].item())
  return scores

In [52]:
similarity_scores = generate_similarity_scores(sample_test_df, similarity_comparison_model)

sample_test_df['similarity_scores_queries'] = similarity_scores

In [53]:
sample_test_df.head()

,question,cypher,database,schema,text,model_generated_cypher_queries,generated_queries_match,similarity_scores_queries
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE ...,False,0.923348
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,False,0.994366
3914,Show the first 5 movies that were originally t...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)\nWHERE m.original_title <> m.t...,False,0.966024
3656,Name the top 5 movies with the longest runtime.,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",neoflix,Node properties:\n- **Movie**\n - `average_vo...,<|begin_of_text|><|start_header_id|>system<|en...,"MATCH (m:Movie)\nRETURN m.title, m.runtime\nOR...",True,1.000000
6124,What are the top 5 movies with the most distin...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,recommendations,Node properties:\n- **Movie**\n - `url`: STRI...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWITH m...,False,0.966013


In [54]:
sample_test_df.similarity_scores_queries.describe()

,similarity_scores_queries
count,30.000000
mean,0.927028
std,0.096171
min,0.541127
25%,0.925336
50%,0.948403
75%,0.980426
max,1.000000


## Let's set a threshold of 80% to consider a Cypher statement as correct.

In [59]:
threshold = 0.85

sample_test_df['generated_queries_match'] = sample_test_df['similarity_scores_queries'].apply(lambda x: True if x >= threshold else False)
sample_test_df.head(2)

,question,cypher,database,schema,text,model_generated_cypher_queries,generated_queries_match,similarity_scores_queries
1720,Which 5 countries have the most filings involv...,MATCH (f:Filing)-[:BENEFITS|:CONCERNS|:ORIGINA...,fincen,Node properties:\n- **Country**\n - `location...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (c:Country)-[:FILED]->(f:Filing)\nWHERE ...,True,0.923348
748,Which organizations are mentioned in articles ...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,companies,Node properties:\n- **Person**\n - `name`: ST...,<|begin_of_text|><|start_header_id|>system<|en...,MATCH (a:Article)-[:MENTIONS]->(o:Organization...,True,0.994366


In [60]:
sample_test_df['generated_queries_match'].value_counts(normalize=True)

,proportion
generated_queries_match,
True,0.9
False,0.1


## Model Evaluation Results - Accuracy: 90%:

- The fine-tuned model demonstrated strong performance, generating correct Cypher queries with 90% accuracy.
- With only 10% of queries being incorrect, the model has effectively learned to align with the expected schema and query structure during evaluation.

Example Prompts

In [ ]:
# ### Working Function
# def generate_cypher_query(question):
#   example_prompt = f"""
#   1. Identify and extract the known entities (e.g., movie title) from the user input.
#       2. Determine the unknown entities (e.g., cast, awards, genres, directors) that need to be retrieved from the graph database.
#       3. Use the known entities to generate a Cypher query that retrieves the values of the unknown entities and validate the generated Cypher query entities and relationships with the reference schema to ensure the query is valid.
#       Schema:
#       {graph_schema}
#       Note: Do not include any explanations or apologies in your responses.
#       Do not respond to any questions that might ask for anything other than extracting entities and generating a Cypher query.
#       Do not include any text except the extracted entities and the generated Cypher query.
#       ###Examples:

#       # Example user input: "Which genres does the movie Inception belong to and who directed it?"
#       Output:
#       MATCH (m:Movie {{title: "Inception"}})
#       OPTIONAL MATCH (m)-[:IN_GENRE]->(g:Genre)
#       OPTIONAL MATCH (p:Person)-[:DIRECTED]->(m)
#       RETURN collect(DISTINCT g.name) AS genres, collect(DISTINCT p.name) AS directors

#       # Example user input: "What is the IMDb rating of The Dark Knight, and in which language was it released?"
#       Output:
#       MATCH (m:Movie {{title: "The Dark Knight"}})
#       OPTIONAL MATCH (m)-[:WAS_RELEASED_IN]->(l:Language)
#       RETURN m.imdbRating AS imdbRating, collect(DISTINCT l.name) AS languages

#       # Example user input: "Who acted in the movie Titanic, and what is its tagline?"
#       Output:
#       MATCH (m:Movie {{title: "Titanic"}})
#       OPTIONAL MATCH (p:Person)-[:ACTED_IN]->(m)
#       RETURN collect(DISTINCT p.name) AS cast, m.tagline AS tagline

#       # Example user input: "Did the movie Avatar win any awards, and what is its IMDb rating?"
#       Output:
#       MATCH (m:Movie {{title: "Avatar"}})
#       RETURN m.awards AS awards, m.imdbRating AS imdbRating

#       # Example user input: "Which languages was the movie Parasite released in and who directed it?"
#       Output:
#       MATCH (m:Movie {{title: "Parasite"}})
#       OPTIONAL MATCH (m)-[:WAS_RELEASED_IN]->(l:Language)
#       OPTIONAL MATCH (p:Person)-[:DIRECTED]->(m)
#       RETURN collect(DISTINCT l.name) AS languages, collect(DISTINCT p.name) AS directors

#       # Example user input: "Recommend 3 movies with at least a award and should be in english language"
#       Output:
#       MATCH (m:Movie)-[:WAS_RELEASED_IN]->(l:Language)
#       WHERE l.name = 'English' AND m.awards > 1
#       RETURN m.title
#       LIMIT 3

#       # Example user input: "Recommend 3 movies with imdb rating at least 7 and at least a award and should be in english language"
#       MATCH (m:Movie)-[:WAS_RELEASED_IN]->(l:Language)
#       WHERE l.name = 'English' AND m.awards > 1 and m.imdbRating >= 7.0
#       RETURN m.title
#       LIMIT 3
#   """

#   messages = [
#         {"role": "system", "content": "Given an input question, convert it to a Cypher query. No pre-amble."},
#         {"role": "user", "content": f"""{example_prompt}

#         Question: {question}
#         Cypher query:"""}
#         ]
#   inputs = tokenizer.apply_chat_template(
#       messages,
#       tokenize = True,
#       add_generation_prompt = True, # Must add for generation
#       return_tensors = "pt",
#   ).to("cuda")

#   outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
#   result = tokenizer.batch_decode(outputs)
#   result = result[0].split("Cypher query:")[-1].strip().split("\n\n")[-1].split("<|eot_id|>")[0]
#   print(result)

In [ ]:
# generate_cypher_query("who is the cast for the movie Road House?")